
# Walmart : predict weekly sales

## Company's Description 📇

Walmart Inc. is an American multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores from the United States, headquartered in Bentonville, Arkansas. The company was founded by Sam Walton in 1962.

## Project 🚧

Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## Goals 🎯

The project can be divided into three steps:

- Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
- Part 2 : train a **linear regression model** (baseline)
- Part 3 : avoid overfitting by training a **regularized regression model**

## Scope of this project 🖼️

For this project, you'll work with a dataset that contains information about weekly sales achieved by different Walmart stores, and other variables such as the unemployment rate or the fuel price, that might be useful for predicting the amount of sales. The dataset has been taken from a Kaggle competition, but we made some changes compared to the original data. Please make sure that you're using **our** custom dataset (available on JULIE). 🤓

## Deliverable 📬

To complete this project, your team should: 

- Create some visualizations
- Train at least one **linear regression model** on the dataset, that predicts the amount of weekly sales as a function of the other variables
- Assess the performances of the model by using a metric that is relevant for regression problems
- Interpret the coefficients of the model to identify what features are important for the prediction
- Train at least one model with **regularization (Lasso or Ridge)** to reduce overfitting


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you: 

### Part 1 : EDA and data preprocessing

Start your project by exploring your dataset : create figures, compute some statistics etc...

Then, you'll have to make some preprocessing on the dataset. You can follow the guidelines from the *preprocessing template*. There will also be some specific transformations to be planned on this dataset, for example on the *Date* column that can't be included as it is in the model. Below are some hints that might help you 🤓

 #### Preprocessing to be planned with pandas

 **Drop lines where target values are missing :**
 - Here, the target variable (Y) corresponds to the column *Weekly_Sales*. One can see above that there are some missing values in this column.
 - We never use imputation techniques on the target : it might create some bias in the predictions !
 - Then, we will just drop the lines in the dataset for which the value in *Weekly_Sales* is missing.
 
**Create usable features from the *Date* column :**
The *Date* column cannot be included as it is in the model. Either you can drop this column, or you will create new columns that contain the following numeric features : 
- *year*
- *month*
- *day*
- *day of week*

**Drop lines containing invalid values or outliers :**
In this project, will be considered as outliers all the numeric features that don't fall within the range : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$. This concerns the columns : *Temperature*, *Fuel_price*, *CPI* and *Unemployment*
 


**Target variable/target (Y) that we will try to predict, to separate from the others** : *Weekly_Sales*

 **------------**

 #### Preprocessings to be planned with scikit-learn

 **Explanatory variables (X)**
We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

 - Categorical variables : Store, Holiday_Flag
 - Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, DayOfWeek

### Part 2 : Baseline model (linear regression)
Once you've trained a first model, don't forget to assess its performances on the train and test sets. Are you satisfied with the results ?
Besides, it would be interesting to analyze the values of the model's coefficients to know what features are important for the prediction. To do so, the `.coef_` attribute of scikit-learn's LinearRegression class might be useful. Please refer to the following link for more information 😉 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

### Part 3 : Fight overfitting
In this last part, you'll have to train a **regularized linear regression model**. You'll find below some useful classes in scikit-learn's documentation :
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

**Bonus question**

In regularized regression models, there's a hyperparameter called *the regularization strength* that can be fine-tuned to get the best generalized predictions on a given dataset. This fine-tuning can be done thanks to scikit-learn's GridSearchCV class : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Also, you'll find here some examples of how to use GridSearchCV together with Ridge or Lasso models : https://alfurka.github.io/2018-11-18-grid-search/

## Librairies
---

In [4]:
#Import Librairies

import pandas as pd
import numpy as np
import datetime

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

## DATASET
---

In [5]:
url = 'Walmart_Store_sales.csv'
df = pd.read_csv(url)

In [6]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Comme indiqué dans le sujet, 8 colonnes. La colonne Date, inexploitable en l'état, à exploser. La target sera la colonne Weekly_Sales. Deux colonnes à considérer comme non-numérique : l'ID du store et le Holiday_Flag

In [7]:
# stats
df.describe(include='all')

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [8]:
# % de données manquantes
display(100*df.isnull().sum()/df.shape[0])

Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [9]:
'''from ydata_profiling import ProfileReport
# Créer un profil
profile = ProfileReport(df, title="Rapport EDA", explorative=True)

# Afficher dans un Jupyter Notebook
profile.to_notebook_iframe()

# Exporter en HTML
profile.to_file("rapport_eda.html")'''

'from ydata_profiling import ProfileReport\n# Créer un profil\nprofile = ProfileReport(df, title="Rapport EDA", explorative=True)\n\n# Afficher dans un Jupyter Notebook\nprofile.to_notebook_iframe()\n\n# Exporter en HTML\nprofile.to_file("rapport_eda.html")'

voir l'interet de ce genre de fichier EDA

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


150 lignes avec des données manquantes. 

## Cleaning
---

La consigne de l'exercice nous permet ne supprimer quelques données sans besoin d'analyse plus spécifique :
Nous allons donc : 
- supprimer les valeurs manquantes de la target, 
- considérer comme outliers les valeurs situées au-delà de 3 écarts-types de la moyenne : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$. Cela concerne les colonnes : Temperature, Fuel_price, CPI and Unemployment

In [11]:
#supprimer les valeurs manquantes de la target
df.dropna(subset=['Weekly_Sales'], inplace=True)

# voir si il y a des valeurs manques sur "weekly_sales"
print(df['Weekly_Sales'].isnull().sum())

0


Verifions les outliers

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Créer une figure avec une grille de 2x3
fig = make_subplots(rows=2, cols=3, subplot_titles=('Temperature', 'Fuel Price', 'CPI', 'Unemployment'))

# Ajouter les boîtes à moustaches à la figure
fig.add_trace(go.Box(y=df['Temperature'], name='Temperature'), row=1, col=1)
fig.add_trace(go.Box(y=df['Fuel_Price'], name='Fuel Price'), row=1, col=2)
fig.add_trace(go.Box(y=df['CPI'], name='CPI'), row=1, col=3)
fig.add_trace(go.Box(y=df['Unemployment'], name='Unemployment'), row=2, col=1)

# Mise en page
fig.update_layout(height=600, width=800, title_text="Distribution des variables")

# Afficher la figure
fig.show()

Bien que l'énoncé évoquait des outliers dans les colonnes "Temperature", "Fuel_price", "CPI" and "Unemployment", seul employement est réellement concernée.

In [13]:
df.shape

(136, 8)

Traitement des Outliers et des valeurs nulles. Commencons par les outliers conformement à la consigne

In [14]:
columns_to_check = ['Unemployment', 'Temperature', 'Fuel_Price']  # Exemple de colonnes
threshold = 3.0  # Nombre d'écart-types pour détecter les outliers

for column in columns_to_check:
    # Calculer la moyenne et l'écart-type pour chaque colonne
    mean = df[column].mean()
    std = df[column].std()
    
    # Identifier les outliers pour la colonne
    outlier_mask = (df[column] < mean - threshold * std) | (df[column] > mean + threshold * std)
    
    # Supprimer les outliers
    df = df[~outlier_mask]

In [15]:
df.shape

(131, 8)

Passons aux valeurs manquantes. Nous avons une valeur de CPI et d'Unemployment. Si il s'agit de taux nationaux, serait-il possible de retrouver des dates manquantes en rapprochant une valeur CPI ou Unemployement identiques ? Essayons

In [16]:
# voir les lignes sans date avec leurs valeurs de "CPI" et "Unemployment"
df[df['Date'].isnull()][['CPI', 'Unemployment']]

,CPI,Unemployment
3,214.556497,7.346
9,224.132020,6.833
17,131.527903,9.202
34,214.929625,NaN
42,222.439015,6.908
65,127.191774,8.744
81,221.434215,5.943
82,223.917015,6.833
83,135.873839,7.806
86,131.037548,6.235


Travaillons sur le CPI et comparons le nombre de valeurs identiques

In [17]:
# Compter le nombre total de valeurs hors NaN dans CPI
num_total_cpi = df['CPI'].notna().sum()

# Identifier les doublons (valeurs avec plus d'une occurrence)
cpi_counts = df['CPI'].value_counts(dropna=True)
duplicates = cpi_counts[cpi_counts > 1]

# Résultat
print(f"Nombre total de valeurs hors NaN dans CPI : {num_total_cpi}")
print(f"Doublons trouvés dans CPI :\n{duplicates}")

Nombre total de valeurs hors NaN dans CPI : 120
Doublons trouvés dans CPI :
CPI
126.7934    2
Name: count, dtype: int64


2 valeurs identiques sur 120 pour le CPI...oublions cette idée et supprimons les valeurs manquantes de la colonne 'Date"

In [18]:
# Supprimer les date vides
df.dropna(subset=['Date'], inplace=True)

In [19]:
df.shape

(113, 8)

Corrigeons les valeurs NaN dans Holiday_Flag. il est possible d'extraire les valeurs NaN et de les comparer et via la bibliothèque "holidays"

In [20]:
# montrer les lignes sans Holiday_Flag
df[df['Holiday_Flag'].isnull()]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
15,6.0,30-04-2010,1498080.16,NaN,68.91,2.780,211.894272,7.092
43,7.0,26-08-2011,629994.47,NaN,57.60,3.485,194.379637,8.622
48,1.0,05-08-2011,1624383.75,NaN,91.65,3.684,215.544618,7.962
53,14.0,25-03-2011,1879451.23,NaN,41.76,3.625,184.994368,8.549
73,1.0,27-08-2010,1449142.92,NaN,85.22,2.619,211.567306,7.787
90,9.0,09-07-2010,485389.15,NaN,78.51,2.642,214.656430,6.442
118,9.0,18-06-2010,513073.87,NaN,82.99,2.637,215.016648,6.384
136,4.0,08-07-2011,2066541.86,NaN,84.59,3.469,129.112500,5.644


In [21]:
#Export un csv avec uniquement la colonne Date pour les colonnes sans Holiday_Flag
df[df['Holiday_Flag'].isnull()]['Date'].to_csv('src/missing_holiday_flag.csv', index=False)


In [22]:
import holidays

# Charger votre fichier CSV
df_holy = pd.read_csv('src/missing_holiday_flag.csv')

# Convertir la colonne de dates en format datetime
df_holy['Date'] = pd.to_datetime(df_holy['Date'])

# Obtenez les jours fériés des États-Unis pour les années 2010 et 2011
us_holidays = holidays.US(years=[2010, 2011])

# Vérifiez si chaque date correspond à un jour férié
df_holy['Is_Holiday'] = df_holy['Date'].dt.date.isin(us_holidays.keys())

# Afficher les dates fériées
holidays_df = df_holy[df_holy['Is_Holiday'] == True]
print(holidays_df)

Empty DataFrame
Columns: [Date, Is_Holiday]
Index: []


C:\Users\dsgat\AppData\Local\Temp\ipykernel_41716\4235990625.py:7: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



Aucune date sans Holidays-Flag ne correspond à un jour férié national. Passons les valeurs de NaN à 0

In [23]:
# remplacer les valeurs NaN dans Holiday_Flag par 0
df['Holiday_Flag'] = df['Holiday_Flag'].fillna(0)
# vérification
df[df['Holiday_Flag'].isnull()]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment


In [24]:
df.shape

(113, 8)

Finissons par la création des colonnes liées à 'Date' et la suppression de la colonnes

In [25]:
# Créer les colonnes Year, Month, Day, DayOfWeek
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df["DayOfWeek"] = df["Date"].dt.day_of_week

In [26]:
# supprimer la colonne date
df = df.drop("Date", axis=1)

In [27]:
df.shape

(113, 11)

Maintenant qu'il est plus propre, analysons le dataset

# EDA
---

In [28]:
#Moyenne des ventes totales
rounded_mean = round(df['Weekly_Sales'].mean(), 2)
print(rounded_mean)

1267414.77


In [29]:
#Moyenne des ventes par magasin
mean_sales_by_store = df.groupby('Store')['Weekly_Sales'].mean().round(2)
print(mean_sales_by_store)

Store
1.0     1550100.94
2.0     1982229.07
3.0      403353.32
4.0     2173758.98
5.0      294398.75
6.0     1551123.58
7.0      536664.40
8.0      888754.13
9.0      506887.40
10.0    1854847.71
11.0    1757242.51
13.0    1997235.41
14.0    2092878.41
15.0     642282.06
16.0     515317.77
17.0     841507.31
18.0    1151981.77
19.0    1400615.22
20.0    1941521.13
Name: Weekly_Sales, dtype: float64


In [30]:
# le total de vente par magasin
total_sales_by_store = df.groupby('Store')['Weekly_Sales'].sum().round(2)
print(total_sales_by_store)

Store
1.0     12400807.53
2.0     11893374.43
3.0      4033533.19
4.0     13042553.90
5.0      2060791.26
6.0      9306741.48
7.0      3756650.81
8.0      5332524.79
9.0      2027549.60
10.0     5564543.12
11.0     1757242.51
13.0    17975118.68
14.0    18835905.73
15.0     1926846.17
16.0     2061271.09
17.0     4207536.54
18.0     8063872.40
19.0    11204921.72
20.0     7766084.51
Name: Weekly_Sales, dtype: float64


In [31]:
# le total de vente par magasin
total_sales_by_store = df.groupby('Store')['Weekly_Sales'].sum().round(2)
print(total_sales_by_store)

Store
1.0     12400807.53
2.0     11893374.43
3.0      4033533.19
4.0     13042553.90
5.0      2060791.26
6.0      9306741.48
7.0      3756650.81
8.0      5332524.79
9.0      2027549.60
10.0     5564543.12
11.0     1757242.51
13.0    17975118.68
14.0    18835905.73
15.0     1926846.17
16.0     2061271.09
17.0     4207536.54
18.0     8063872.40
19.0    11204921.72
20.0     7766084.51
Name: Weekly_Sales, dtype: float64


In [32]:
corr = df.corr()
fig = px.imshow(corr)

# Ajouter les valeurs de corrélation dans les carrés
for y in range(corr.shape[0]):
    for x in range(corr.shape[1]):
        fig.add_annotation(x=x, y=y, 
                           text=str(round(corr.iloc[y, x], 2)), 
                           showarrow=False, 
                           font=dict(color="black"))


# Ajuster la mise en page pour une meilleure lisibilité
fig.update_layout(
        xaxis=dict(side="top"),
        width=800,  # Largeur de la figure en pixels
        height=600)  # Hauteur de la figure en pixels

fig.show()

## Analyse des corrélations

L'analyse de la matrice de corrélation révèle plusieurs insights importants :

1. Corrélations avec les ventes hebdomadaires (Weekly_Sales) :
   - Store (0.13) : Faible corrélation positive, suggérant une légère variation des performances selon les magasins
   - Unemployment (0.18) : Corrélation positive inattendue, potentiellement due à d'autres facteurs économiques
   - Temperature (-0.19) : Impact négatif modéré, les ventes diminuent légèrement par temps chaud
   - CPI (-0.36) : La corrélation négative la plus forte, indiquant une sensibilité des consommateurs aux prix

2. Corrélations entre variables explicatives :
   - Fuel_Price et Year (0.83) : Forte corrélation indiquant une multicolinéarité à prendre en compte
   - Les variables temporelles (Month, Day, DayOfWeek) ont des corrélations faibles avec les autres variables
   - Holiday_Flag montre des corrélations faibles avec toutes les variables, suggérant une indépendance relative

Ces observations suggèrent l'importance de la régularisation pour gérer la multicolinéarité. Potentiellement , possibilité d'exclure les variables temporelles peu corrélées pour éviter le bruit ?

In [33]:
# graph de l'impact de la température sur les ventes dans un petit graphique
fig = px.scatter(df, x='Temperature', y='Weekly_Sales', trendline='ols')
fig.show()



   - Relation négative modérée (cf pente)
   - Grande dispersion des points suggérant d'autres facteurs influents
   - Les ventes semblent plus volatiles aux températures moyennes


In [34]:
# graph de l'impact du taux de chomage sur les ventes
fig = px.scatter(df, x='Unemployment', y='Weekly_Sales', trendline='ols')
fig.show()


   - Relation positive faible mais significative (pente)
   - La dispersion augmente avec le taux de chômage
   - Potentielle relation non-linéaire à explorer

In [35]:
# graph de l'impact de l'inflation sur les ventes
fig = px.scatter(df, x='CPI', y='Weekly_Sales', trendline='ols')
fig.show()

   - Relation négative significative (pente)
   - Deux clusters distincts (raisons ?)
   - Impact plus prononcé sur les ventes élevées

In [36]:
#Graph de l'impact du prix du fuel sur les ventes
fig = px.scatter(df, x='Fuel_Price', y='Weekly_Sales', trendline='ols')
fig.show()

   - Relation négative très faible (pente)
   - Grande dispersion indiquant un impact limité
   - Possible effet indirect via d'autres variables économiques

In [37]:
#graph montant des ventes par mois
fig = px.bar(df.groupby('Month')['Weekly_Sales'].sum().reset_index(), x='Month', y='Weekly_Sales')
fig.show()

Pics de ventes :
   - Février (18.3M$) : soldes d'hiver ?
   - Juin (16.7M$) : Début de la période estivale
   - Décembre (16.0M$) : Période des fêtes

Creux saisonniers :
   - Septembre (6.0M$) : Rentrée scolaire
   - Octobre (6.5M$) : Période transitoire

# PREPROCESSING
---

In [38]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = df.drop(target_variable, axis = 1)
y = df[target_variable]

Separating labels from features...


In [39]:
# Division du dataset en test et train set
print("Division en set de train et de test ...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("...Terminé.")
print()

Division en set de train et de test ...
...Terminé.



Il reste des valeurs manquantes non traité. Ayant supprimé les outliers, nous faisons le choix de la métrique "median" pour le Simpleimputer. Autrement mean aurait pu être plus adapté car moins sensible aux valeurs extrèmes de la témpérature ou des pics de prix sur le fuel par exemple

In [40]:
# Designation des features catégorielles et numériques
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'DayOfWeek']
categorical_features = ['Store', 'Holiday_Flag']

# Transformation des features catégorielles et numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())  
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  
])

# Combination de tout le preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        
    ])

# Création du pipeline avec le préprocesseur et le modèle
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression()) 
])

In [41]:
# Entrainement du modèle sur le set d'entraînement
model_pipeline.fit(X_train, y_train)

# Prédictions sur l'ensemble d'entraînement et l'ensemble de test
y_train_pred = model_pipeline.predict(X_train)
y_test_pred = model_pipeline.predict(X_test)

In [42]:
# Calculer les scores
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Calcul du MAPE
erreurs_relatives_train = abs(y_train - y_train_pred) / y_train
mape_train = np.mean(erreurs_relatives_train) * 100

erreurs_relatives_test = abs(y_test - y_test_pred) / y_test
mape_test = np.mean(erreurs_relatives_test) * 100

# Afficher les résultats
print("R2 score on training set : {:.3f}".format(r2_train))
print("R2 score on test set : {:.3f}".format(r2_test))

print(f"MAE sur l'ensemble d'entraînement : {mae_train/1e6:.3f} millions $")
print(f"MAE sur l'ensemble de test : {mae_test/1e6:.3f} millions $")

print(f"MAPE sur l'ensemble d'entraînement : {mape_train:.3f}%")
print(f"MAPE sur l'ensemble de test : {mape_test:.3f}%")

R2 score on training set : 0.972
R2 score on test set : 0.949
MAE sur l'ensemble d'entraînement : 0.084 millions $
MAE sur l'ensemble de test : 0.119 millions $
MAPE sur l'ensemble d'entraînement : 8.414%
MAPE sur l'ensemble de test : 14.415%


## AMELIORATION DU SCORE : RIDGE / LASSO et GRIDSEARCHCV
---

In [43]:
# Fonction d'optimisation
def optimize_model(model_type, param_grid, X_train, y_train):
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model_type)
    ])
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, y_train)
    return grid_search

# Définition des paramètres et optimisation
param_grid = {'regressor__alpha': [0.1, 1, 10, 100, 1000]}

# Optimisation des modèles Ridge et Lasso
ridge_grid_search = optimize_model(Ridge(), param_grid, X_train, y_train)
lasso_grid_search = optimize_model(Lasso(), param_grid, X_train, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres Ridge:", ridge_grid_search.best_params_)
print("Meilleurs paramètres Lasso:", lasso_grid_search.best_params_)

c:\Users\dsgat\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.091e+09, tolerance: 3.300e+09

c:\Users\dsgat\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.969e+09, tolerance: 3.184e+09

c:\Users\dsgat\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.241e+09, tolerance: 2.798e+09

c:\Users\dsgat\anaconda3\Lib\site-packages\sklearn\linear_model\_co

Meilleurs paramètres Ridge: {'regressor__alpha': 0.1}
Meilleurs paramètres Lasso: {'regressor__alpha': 1000}


In [44]:
# Fonction d'évaluation
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculs des métriques
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    erreurs_relatives_train = abs(y_train - y_train_pred) / y_train
    mape_train = np.mean(erreurs_relatives_train) * 100
    
    erreurs_relatives_test = abs(y_test - y_test_pred) / y_test
    mape_test = np.mean(erreurs_relatives_test) * 100
    
    # Affichage des résultats
    print(f"\nRésultats pour {model_name}:")
    print(f"R² train: {r2_train:.3f}")
    print(f"R² test: {r2_test:.3f}")
    print(f"MAE train: {mae_train/1e6:.3f} millions $")
    print(f"MAE test: {mae_test/1e6:.3f} millions $")
    print(f"MAPE train: {mape_train:.3f}%")
    print(f"MAPE test: {mape_test:.3f}%")

# Évaluation des deux modèles
evaluate_model(ridge_grid_search.best_estimator_, X_train, X_test, y_train, y_test, "Ridge")
evaluate_model(lasso_grid_search.best_estimator_, X_train, X_test, y_train, y_test, "Lasso")


Résultats pour Ridge:
R² train: 0.971
R² test: 0.940
MAE train: 0.085 millions $
MAE test: 0.126 millions $
MAPE train: 8.525%
MAPE test: 14.409%

Résultats pour Lasso:
R² train: 0.971
R² test: 0.959
MAE train: 0.085 millions $
MAE test: 0.108 millions $
MAPE train: 8.262%
MAPE test: 13.728%


In [46]:
# créer un Plot coefficients
ridge_coefs = ridge_grid_search.best_estimator_.named_steps['regressor'].coef_
lasso_coefs = lasso_grid_search.best_estimator_.named_steps['regressor'].coef_

# Créer un DataFrame avec les coefficients
coefs_df = pd.DataFrame({
    'Feature': numeric_features + list(lasso_grid_search.best_estimator_.named_steps['preprocessor'].transformers_[1][1].named_steps['encoder'].get_feature_names_out(categorical_features)),
    'Ridge Coefficients': ridge_coefs,
    'Lasso Coefficients': lasso_coefs
})

# Afficher les coefficients
coefs_df

# Créer un graphique pour les coefficients
fig = make_subplots(rows=2, cols=1, subplot_titles=('Ridge Coefficients', 'Lasso Coefficients'))

# Ajouter les barres pour les coefficients Ridge
fig.add_trace(go.Bar(x=coefs_df['Feature'], y=coefs_df['Ridge Coefficients'], name='Ridge Coefficients'), row=1, col=1)

# Ajouter les barres pour les coefficients Lasso
fig.add_trace(go.Bar(x=coefs_df['Feature'], y=coefs_df['Lasso Coefficients'], name='Lasso Coefficients'), row=2, col=1)

# Mise en page
fig.update_layout(height=800, width=800, title_text="Coefficients des modèles Ridge et Lasso")

# Afficher la figure
fig.show()

# Conclusion

## Performance des modèles
Les trois modèles (Régression linéaire, Ridge et Lasso avec GridSearch) montre des performances relativement similaires :


| Modèle | R² Train | R² Test | MAE Train (M$) | MAE Test (M$) | MAPE Train (%) | MAPE Test (%) |
|--------|----------|---------|---------------|--------------|---------------|--------------|
| Baseline (Linéaire) | 0,972 | 0,949 | 0,084 | 0,119 | 8,414 | 14,415 |
| Ridge | 0,971 | 0,940 | 0,085 | 0,126 | 8,525 | 14,409 |
| Lasso | 0,971 | 0,950 | 0,085 | 0,108 | 8,262 | 13,728 |

Observations :
- Lasso légèrement supérieur
- Performances très proches
- MAE test Lasso : meilleur (0,108 M$)

mais avec des nuances :

1. Régression linéaire (baseline) :
   - R² test = 0.949
   - Performance solide

2. Ridge (α = 0.1) :
   - R² test = 0.940
   - Légère dégradation des performances mais meilleure stabilité

3. Lasso (α = 1000) :
   - R² test = 0.950
   - Meilleure performance en test

Le modèle Lasso avec un alpha de 1000 se révèle être le plus performant avec :
- Un R² de 0.95 sur le jeu de test, indiquant une excellente capacité de prédiction
- Une MAE test légèrement inférieure aux autres modèles
- Une meilleure capacité de généralisation avec un écart train/test raisonnable

## Insights Business

### Variables clés influençant les ventes
1. Impact des magasins (Store) :
   - Forte variation des performances entre magasins
   - Les stores 14, 2 et 20 montrent les coefficients les plus positifs
   - Les stores 5, 16 et 9 ont les coefficients les plus négatifs

2. Facteurs temporels :
   - Le mois (Month) a un impact positif significatif sur les ventes
   - Les ventes sont plus élevées en février et juin
   - La saisonnalité est un facteur important à considérer

3. Facteurs économiques :
   - Impact négatif du chômage (Unemployment : -69,813)
   - Corrélation négative avec le CPI, suggérant une sensibilité aux prix
   - Impact modéré du prix du carburant (Fuel_Price : -41,088)

3. Performance du Modèle :
   - Modèle prédictif avec R² de 0,95
   - Capture 95% des variations structurelles des ventes
   - Écart de prédiction de ±14% reflétant des variations locales et conjoncturelles non prévisibles

### Recommandations Business
1. Optimisation par magasin :
   - Analyser les pratiques des magasins performants (14, 2, 20)
   - Mettre en place des plans d'action spécifiques pour les magasins sous-performants
   - Standardiser les meilleures pratiques

2. Gestion saisonnière :
   - Adapter les stocks en fonction des pics de vente mensuels
   - Optimiser les promotions pendant les périodes creuses
   - Ajuster les ressources humaines selon la saisonnalité

3. Stratégie prix :
   - Surveiller l'impact de l'inflation (CPI) sur les ventes
   - Adapter la politique tarifaire en fonction des indicateurs économiques
   - Développer des programmes de fidélisation pour les périodes économiques difficiles

## Améliorations Possibles
   - Tester des modèles non-linéaires (Random Forest, XGBoost)
   - Créer des features d'interaction
   - Implémenter une validation croisée temporelle
   - Explorer des techniques de feature engineering plus avancées

Cette analyse fournit à Walmart des outils concrets pour optimiser ses ventes et adapter sa stratégie en fonction des différents facteurs influençant la performance des magasins.